In [ ]:
library(igraph)

First example from [igraph graph_from_adjacency_matrix() documentation](http://search.r-project.org/library/igraph/html/graph_from_adjacency_matrix.html)

In [ ]:
adjm <- matrix(sample(0:1, 100, replace=TRUE, prob=c(0.9,0.1)), nc=10)
g1 <- graph_from_adjacency_matrix( adjm )
plot(g1)

Explore bipartite adjacency construction using igraph.  Follows model of example https://rpubs.com/pjmurphy/317838

Create a random relation R that maps set A={1,2,3} to set B={1,2,3,4}.  This is easily done using a 3x4 matrix with the three rows representing the elements of set A and the four columns representing the elements of set B. The matrix is randomly populated with ones to represent the mapping between the elements of both sets.

(A side note: Don't be confused by the use of similar numbers as elements in both sets.  The numbers only represent distinct set elements and don't carry of any of their common numeric meaning.  The one in set A does not equal the one in set B.  Only the relation defines connections between elements of the sets. For example if R defines a relation between the 1 in set A and the 1 in set B then there will be a one in the 1,1 cell of the connectivity matrix that represents R.  Otherwise the cell value is zero.)

In [ ]:
R <- matrix(sample(0:1, 12, replace=TRUE, prob=c(0.3,0.7)), nc=4)
R

A relation implicitly describes a [bipartite graph](https://en.wikipedia.org/wiki/Bipartite_graph). This means the relation maps a distinct set of elements in A to a distinct set of elements in B and there are no relations between elements of the same set. In graph form, the elements of both sets A and B are treated as vertices, or nodes, in the graph.  The relation R is represented as the edges connecting those nodes.

A graph can be conveniently represented as an adjacency matrix. Adjacency matricies are square.  The rows and columns represent the full set of vertices in the graph and the cell values describe the edges between them.  A binary value {0,1} represents connectivity, the presense or absense of an edge.

Representing a relation as a graph means treating the combined elements of sets A and B as the vertices of the graph.  In the above relation the three elements of set A and four elements of set B combine to create seven graph verticies. This results in a 7x7 matrix with cells populated based on the edges connecting the vertices. To fit with the adjacency matrix structure, we will keep set elements of A as {1, 2, 3} and rename the elements of set B as {4, 5, 6, 7}.  The relation remains the same, we are just keeping the element "names" distinct across the sets to make it easier to work with them as distinct vertices of the graph.

The adjacency matrix is square because all the vertices are repeated in both the rows and columns. An adjacency matrix can describe a directed graph, where the edges have a direction and point from one vertex to another. A one in the cell 3,4 means there is an edge from vertex three to vertex four.  If there is also a one in cell 4,3 then there is an edge pointing back from vertex four to vertex three. A matrix that has the same value for cell i,j and cell j,i is symetric.  It represents a directed graph that has edges pointing in both directions or an undirected graph, where the the edge direction doesn't matter  In the case of an undirected graph, we technically only need to record the upper (or lower) triangle since the matrix is symetric but we won't worry about that space efficiency concern in these examples.

In order to visualize our relation as a graph we need to turn it into an adjacency matrix.  Because the [relation implicitly decribes a bipartite graph, it is easy to convert the relation to an adjaceny matrix](https://en.wikipedia.org/wiki/Adjacency_matrix#Of_a_bipartite_graph).\

![](https://wikimedia.org/api/rest_v1/media/math/render/svg/153a4f46f7639209d3d477aa715558c879b00319)


In our example above, the 7x7 adjaceny matrix has three rows and columns that represent elements of set A, numbered {1, 2, 3}, and four rows and columns that represent the elements of set B, now numbered {4, 5, 6, 7}.  Both of the square 3x3 and 4x4 submatrices in the upper left and lower right are zero because there are no intra-set connections.  The 3x4 submatrix in the upper right quadrant, the position of "B" in this equation, directly contains our example relation R. (The "B" in this equation stands for "Biadjacency martrix" and is unrelated to name of our set "B" above).  The remaining 4x3 submatrix in the lower left quadrent is simply the transpose of relation.  This makes our adjaceny matrix symetric and implies the edges are bidirectional, or if we don't care about direction, undirected.

Convert the relation to an adjaceny matrix.  

In [ ]:
?V

In [ ]:
t(R)

In [ ]:
z1 <- matrix(c(0), nr=3, nc=3 )
z1

In [ ]:
cbind(z1, R)

In [ ]:
cbind(t(R), matrix(c(0), nr=4, nc=4))

In [ ]:
relation2adjacency <- function(R) {
    return(rbind(cbind(matrix(c(0), nr=dim(R)[1], nc=dim(R)[1]), R), cbind(t(R), matrix(c(0), nr=dim(R)[2], nc=dim(R)[2]))))
}

In [ ]:
relation2adjacency(R)

Construct the adjacency matrix from the relation R.  We use the transpose of the relation, t(R), to populate the lower left quadrant making the matrix symetric and allowing it to be treated as an undirected graph.

In [ ]:
bipartite_adjm <- rbind(cbind(z1, R), cbind(t(R), matrix(c(0), nr=4, nc=4)))

Construct a second adjacency matrix from R.  We use a zero matrix to populate the lower left quadratant leaving the matrix un-symetric and allowing it to be treated as a directed graph.

In [ ]:
bipartite_adjm_directed <- rbind(cbind(z1, R), cbind(matrix(c(0), nr=4, nc=3), matrix(c(0), nr=4, nc=4)))

Convert the adjacency matrices to graphs.  The mode parameter only controls if the edges are drawn with arrows or without, it does not change the nature of the adjacency matrix we constructed.

In [ ]:
g <- graph_from_adjacency_matrix( bipartite_adjm, mode="undirected" )

In [ ]:
gd <- graph_from_adjacency_matrix( bipartite_adjm_directed, mode="directed" )

Test if the graph represents a bipartite graph. The result, $res, is true because we explicitly constructed bipartite adjacency matricies from the relation above.  The type attribute tells us to which set the vertices belong.  The FALSE are set A and the TRUE are set A.

In [ ]:
bipartite.mapping(g)

In [ ]:
V(g)

In [ ]:
E(g)

Plot the undirected version of the graph.

In [ ]:
plot(g)

Plot the directed version of the graph.  Not the edges now have errors.

In [ ]:
plot(gd)

With directed edges its easier to recognize that the vertices are two distinct sets and that there are only edges between those sets.  It would be better still to have the vertices from different sets look different.  Let's change the colors and shapes.

In [ ]:
V(g)$type <- bipartite_mapping(g)$type

In [ ]:
plot(g, vertex.label.cex = 0.9, vertex.label.color = "black")

In [ ]:
V(g)$color <- ifelse(V(g)$type, "lightblue", "salmon")
V(g)$shape <- ifelse(V(g)$type, "circle", "square")
E(g)$color <- "lightgray"
  
plot(g, vertex.label.cex = 0.8, vertex.label.color = "black")

In [ ]:
?igraph

In [ ]:
V(g)$label.color <- "black" ##ifelse(V(g)$type, "black", "white")
## V(g)$label.font <-  2
V(g)$label.cex <- 1 ##ifelse(V(g)$type, 0.8, 1.2)
## V(g)$label.dist <-0
V(g)$frame.color <-  "gray"
V(g)$size <- 18

plot(g, layout = layout_with_graphopt)

Using a bipartite layout with the vertices different shapes and different colors, we can now easily read the graph. We don't even need directed edges. 

In [ ]:
plot(g, layout=layout.bipartite, vertex.size=7, vertex.label.cex=0.6)

Now let's talk about relation composition and see how to represent that as part of our graph.

Our example will use a new relation S that represents a secondary relationship between elements of set B, a mapping from set B onto elements of set B.  This means the relation is between the four elements so the matrix will be 4x4.

In [ ]:
S <- matrix(sample(0:1, 16, replace=TRUE, prob=c(0.6,0.4)), nc=4)
S

The composition of the relations can be computed with a matrix multiplication of the two sets.  

In [ ]:
R %*% S

In [ ]:
S

In [ ]:
Sa = relation2adjacency(S)

In [ ]:
relation2adjacency

In [ ]:
Sa = rbind(cbind(matrix(c(0), nr=dim(R)[1], nc=dim(R)[1]), R), cbind(t(R), S))

In [ ]:
s = graph_from_adjacency_matrix( Sa, mode="undirected", diag=FALSE )

In [ ]:
plot(s)

In [ ]:
library(ggplot2)

In [ ]:
as.data.frame(Sa)

In [ ]:
library(ape)

In [ ]:
library(help=ape)

In [ ]:
plot(rtree(20))